In [1]:
import torch

random_b0 = torch.rand((16, 1, 128, 178, 115))
random_residual = torch.rand((16, 1, 128, 178, 115))

print(random_b0.shape)
print(random_residual.shape)

torch.Size([16, 1, 128, 178, 115])
torch.Size([16, 1, 128, 178, 115])


In [2]:
from training import *

b0_cube = grab_cube_around_voxel(image=random_b0, voxel_coordinates=[5, 5, 5], kernel_size=16)
b0_cube = torch.from_numpy(b0_cube).float()

res_cube = grab_cube_around_voxel(image=random_residual, voxel_coordinates=[5, 5, 5], kernel_size=8)
res_cube = torch.from_numpy(res_cube).float()

injection_center = torch.rand((16, 3, 16, 16, 16))
image_coordinates = torch.rand((16, 3, 16, 16, 16))

print(b0_cube.shape)
print(res_cube.shape)
print(injection_center.shape)
print(image_coordinates.shape)

torch.Size([16, 1, 32, 32, 32])
torch.Size([16, 1, 16, 16, 16])
torch.Size([16, 1, 1, 1, 3])
torch.Size([16, 1, 1, 1, 3])


In [3]:
from models import *
model = Attention_UNet()

In [4]:
output = model(b0_cube, injection_center, image_coordinates)

print("output shape is: ", output.shape)

conv1:  torch.Size([16, 16, 32, 32, 32])
maxpool1:  torch.Size([16, 16, 16, 16, 16])
conv2:  torch.Size([16, 32, 16, 16, 16])
maxpool2:  torch.Size([16, 32, 8, 8, 8])
conv3:  torch.Size([16, 64, 8, 8, 8])
maxpool3:  torch.Size([16, 64, 4, 4, 4])
conv4:  torch.Size([16, 128, 4, 4, 4])
maxpool4:  torch.Size([16, 128, 2, 2, 2])
center:  torch.Size([16, 256, 2, 2, 2])
gating:  torch.Size([16, 256, 2, 2, 2])
g_conv4:  torch.Size([16, 128, 4, 4, 4])
up4:  torch.Size([16, 128, 4, 4, 4])
g_conv3:  torch.Size([16, 64, 8, 8, 8])
up3:  torch.Size([16, 64, 8, 8, 8])
g_conv2:  torch.Size([16, 32, 16, 16, 16])
up2:  torch.Size([16, 32, 16, 16, 16])
up1:  torch.Size([16, 16, 32, 32, 32])
dsv4:  torch.Size([16, 1, 32, 32, 32])
dsv3:  torch.Size([16, 1, 32, 32, 32])
dsv2:  torch.Size([16, 1, 32, 32, 32])
dsv1:  torch.Size([16, 1, 32, 32, 32])
final:  torch.Size([16, 1, 32, 32, 32])


RuntimeError: Given groups=1, weight of size [3, 3, 3, 3, 3], expected input[16, 1, 1, 1, 3] to have 3 channels, but got 1 channels instead

In [ ]:
import glob
import os
import nibabel as nib

def glob_files(PATH_NAME, file_format):
    INPUT_FILES = []
    for file in glob.glob(os.path.join(PATH_NAME, os.path.join("**", "*.{}".format(file_format))), recursive=True):
        INPUT_FILES.append(file)
    return INPUT_FILES

nii_gz_files = glob_files("/notebooks/model_data_w_resize", "nii.gz")
b0_images = [file for file in nii_gz_files if "b0" in file and "resized" not in file]
print(len(b0_images))


In [42]:
import SimpleITK as sitk

reader = sitk.ImageFileReader()
reader.SetFileName(b0_images[0])
image = reader.Execute()

normalizeFilter = sitk.NormalizeImageFilter()
rescaleFilter = sitk.RescaleIntensityImageFilter()
rescaleFilter.SetOutputMaximum(255)
rescaleFilter.SetOutputMinimum(0)

image = normalizeFilter.Execute(image)
image = rescaleFilter.Execute(image)

array = sitk.GetArrayFromImage(image)
print(array.shape)

image_normalized = sitk.GetImageFromArray(array)

sitk.WriteImage(image_normalized, os.path.join(os.getcwd(), "test2.nii.gz"));

(230, 356, 256)


In [4]:
import nibabel as nib
import numpy as np

image = nib.load(b0_images[0])
data = image.get_fdata()

print(data.shape)

normalized_vector = data / np.linalg.norm(data)

print(normalized_vector.shape)

final_img = nib.Nifti1Image(normalized_vector, image.affine)

nib.save(final_img, os.path.join(os.getcwd(), "nibabel.nii"))

(256, 356, 230)
(256, 356, 230)


In [5]:
path = os.path.join(os.getcwd(), "nibabel.nii")

image = nib.load(path)
data = image.get_fdata()

print(data.shape)

(256, 356, 230)


In [16]:
import numpy as np

path_to_residuals = "/notebooks/tract_residuals/predicted_residuals/epoch_1/image_0.npy"
image0 = np.load(path_to_residuals)
print(image0.shape)

(4, 12, 8, 8, 8, 8)


In [26]:
def to_shape(a, shape):
    y_, x_, z_ = shape
    y, x, z = a.shape
    y_pad = (y_-y)
    x_pad = (x_-x)
    z_pad = (z_-z)
    return np.pad(a,((y_pad//2, y_pad//2 + y_pad%2), 
                     (x_pad//2, x_pad//2 + x_pad%2),
                     (z_pad//2, z_pad//2 + z_pad%2)),
                  mode = 'constant')

# Create random array
random_array = np.random.rand(128, 178, 115)
print(random_array.shape)

(128, 178, 115)


In [27]:
# Define kernel size
kernel_size = 8 * 2

# Get the number of values for each axes that need to be added to fit multiple of kernel
padding_needed = [axis % kernel_size for axis in random_array.shape]
print(padding_needed)

output_shape = []
for i in range(random_array.ndim):
    output_shape.append(random_array.shape[i] + padding_needed[i])
    
print(output_shape)

# Padding the random array to the new shape
random_reshaped = to_shape(random_array, output_shape)
print(random_reshaped.shape)

[0, 2, 3]
[128, 180, 118]
(128, 180, 118)


In [10]:
import nibabel as nib
import numpy as np

path = "/notebooks/model_data_w_resize/dMRI_b0/A10-R01_0028-TT21/DWI_concatenated_b0_resized.nii.gz"
injection_center = "/notebooks/model_data_w_resize/injection_centers/A10-R01_0028-TT21/inj_center.csv"
stream_path = "/notebooks/model_data_w_resize/tckmapped_streamlines/A10-R01_0028-TT21/subtracted_unflipped_resized.nii.gz"

# Load the image
image = nib.load(path)
streamline = nib.load(stream_path)
data = image.get_fdata()
stream_data = streamline.get_fdata()
print(data.shape)
print(stream_data.shape)


(128, 178, 115)
(128, 178, 115)


In [13]:
b0_hemi = data[64:, :, :]
res_hemi = stream_data[64:,:,:]

img = nib.Nifti1Image(b0_hemi, affine=np.eye(4))
img2 = nib.Nifti1Image(res_hemi, affine=np.eye(4))

img_b0 = nib.Nifti1Image(data, affine=np.eye(4))
img_str = nib.Nifti1Image(stream_data, affine=np.eye(4))

nib.save(img, "testingcut.nii")
nib.save(img2, "testingcut_res.nii")
nib.save(img_b0, "ogb0.nii")
nib.save(img_str, "ogres.nii")
